In [1]:
import numpy as np
from scipy import stats
import pickle
import torch
import datetime
from PIL import Image
from matplotlib import pyplot as plt

from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.svm import NuSVC
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
def loadFeature(path):
    f=open(path,'rb')
    feature=[]
    dim=102
    trainFeature=pickle.load(f)
    for i in range(len(trainFeature['face']['mean'])):
        for j in  range(len(trainFeature['face']['mean'][i])):
            fv=np.zeros(dim+1)
            fv[:dim]= trainFeature['face']['mean'][i][j].numpy()
            #fv[2048:4096]= trainFeature['face']['v'][i][j].numpy()
            fv[dim]=1
            feature.append(fv)

    for i in range(len(trainFeature['scramble']['mean'])):
        for j in  range(len(trainFeature['scramble']['mean'][i])):
            fv=np.zeros(dim+1)
            fv[:dim]= trainFeature['scramble']['mean'][i][j].numpy()
            #fv[2048:4096]= trainFeature['scramble']['v'][i][j].numpy()
            fv[dim]=0
            feature.append(fv)
    return feature
        
def loadFeatureWithZ(path):
    f=open(path,'rb')
    feature=[]
    dim=102
    trainFeature=pickle.load(f)
    for i in range(len(trainFeature['face']['z'])):
        for j in  range(trainFeature['face']['z'][i].shape[0]):
            fv=np.zeros(dim+1)
            fv[:dim]= trainFeature['face']['z'][i][j]
            #fv[2048:4096]= trainFeature['face']['v'][i][j].numpy()
            fv[dim]=1
            feature.append(fv)

    for i in range(len(trainFeature['scramble']['z'])):
        for j in  range(trainFeature['scramble']['z'][i].shape[0]):
            fv=np.zeros(dim+1)
            fv[:dim]= trainFeature['scramble']['z'][i][j]
            #fv[2048:4096]= trainFeature['scramble']['v'][i][j].numpy()
            fv[dim]=0
            feature.append(fv)
    return feature

In [18]:
feature = loadFeatureWithZ('./Model_output_Feature/Training_Feature_R108E80_102_AllClass_kKLDloss_with8Z')
feature = np.array(feature)
feature.shape

(75200, 103)

In [19]:
f=shuffle(feature)
t=int(np.round(f.shape[0]*0.7))
train = f[:t,:] 
dim=102

#C_range = np.logspace(-2, 10, 5)
gamma_range = np.linspace(1, 10, 1)
#degree_range = np.arange(5,20,1)
nu_r=np.linspace(0.1, 0.99, 5)
print(nu_r)
#ker= ['rbf', 'sigmoid']
param_grid = dict(nu=nu_r)
#param_grid = {'degree': [5,10,15,20],'nu' : [0.1,0.2,0.3,0.4,0.5]}
#svc = SVC(random_state=13)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1024)
search = GridSearchCV(NuSVC(kernel='sigmoid',random_state=13), param_grid=param_grid, scoring="accuracy", cv=cv,n_jobs=-1,verbose=2)
search.fit(train[:,0:dim] , train[:,dim:dim+1].reshape(train.shape[0],))

[0.1    0.3225 0.545  0.7675 0.99  ]
Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] END .............................................nu=0.1; total time= 8.8min
[CV] END ...........................................nu=0.545; total time=30.4min
[CV] END ............................................nu=0.99; total time=19.3min
[CV] END .............................................nu=0.1; total time= 8.4min
[CV] END ..........................................nu=0.3225; total time=23.3min
[CV] END ..........................................nu=0.7675; total time=27.4min
[CV] END .............................................nu=0.1; total time= 8.5min
[CV] END ...........................................nu=0.545; total time=28.9min
[CV] END ..........................................nu=0.7675; total time=25.2min
[CV] END .............................................nu=0.1; total time= 8.5min
[CV] END ...........................................nu=0.545; total time=28.8min
[CV] END ...

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=1, n_splits=5, random_state=1024),
             estimator=NuSVC(kernel='sigmoid', random_state=13), n_jobs=-1,
             param_grid={'nu': array([0.1   , 0.3225, 0.545 , 0.7675, 0.99  ])},
             scoring='accuracy', verbose=2)

In [23]:
search.best_score_

0.5016907294832827

In [22]:

testset = f[t:,0:dim]
testy = f[t:,dim:dim+1]
result = search.best_estimator_.predict(testset)
print(classification_report(result,testy))

              precision    recall  f1-score   support

         0.0       0.50      0.50      0.50     11414
         1.0       0.49      0.50      0.49     11146

    accuracy                           0.50     22560
   macro avg       0.50      0.50      0.50     22560
weighted avg       0.50      0.50      0.50     22560

